In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Embedding, Flatten, Dense

# Load data
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

# Save the model
model.save('model/emotion_model2.h5')


                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness
Epoch 1/10


D:\Data Engineering Project\ML Project\aiwork\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.3861 - loss: 1.5232 - val_accuracy: 0.7416 - val_loss: 0.8004
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.8879 - loss: 0.3751 - val_accuracy: 0.8156 - val_loss: 0.5605
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9880 - loss: 0.0558 - val_accuracy: 0.8281 - val_loss: 0.5691
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9962 - loss: 0.0201 - val_accuracy: 0.8297 - val_loss: 0.5972
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9963 - loss: 0.0201 - val_accuracy: 0.8194 - val_loss: 0.6588
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9982 - loss: 0.0099 - val_accuracy: 0.8191 - val_loss: 0.6575
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9979 - loss: 0.0090 - val_accuracy: 0.8247 - val_loss: 0.6856
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9978 - loss: 0.0085 - val_accuracy: 0.819

In [3]:
import pickle
# Save the tokenizer
with open('model/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('model/label_encoder.pkl', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load the model and tokenizer for predictions
model = load_model('model/emotion_model2.h5')

with open('model/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('model/label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

# Preprocess the input text
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"

input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
love


In [5]:
# Load the model and tokenizer for predictions
model2 = load_model('model/emotion_model2.h5')

with open('model/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('model/label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

# Preprocess the input text
input_text = "i need to feel the dough to make sure its just perfect"

input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model2.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
joy
